In [2]:
import numpy as np

In [3]:
qubit_map = {'0':np.array([1,0]),'1':np.array([0,1])}

def all_bits_on(bitstring,controls,control_val,control_bitstring=''):
    if control_bitstring=='': control_bitstring=control_val*len(controls)
    for bit in range(len(controls)):
        if bitstring[controls[bit]] != control_bitstring[bit]:
            return False
    return True
def projection_on_the_rest(remaining_bitstring):
    product = np.array(1.)
    for bit in remaining_bitstring:
        product=np.kron(product,np.outer(qubit_map[bit],qubit_map[bit]))
    return product

def int_to_bitstring(i,length):
    bitstring = bin(i)[2:]
    bitlength = len(bitstring)
    if bitlength < length:
        for j in range(length-bitlength):
            bitstring = '0'+bitstring
    return bitstring

def f_cnot(bitstring,control,target):
    if bitstring[control] == '1':
        bitstring = bitstring[:target] + str(1-int(bitstring[target]))+bitstring[1+target:]
    return bitstring

def bitstrings_to_vector(bitstrings):
    if type(bitstrings) == dict:
        num_bits = len(list(bitstrings)[0])
        vec = np.zeros(2**num_bits)
        for bitstring in bitstrings:
            this_vec = np.array(1.)
            for bit in bitstring:
                this_vec = np.kron(this_vec,qubit_map[bit])
            vec = vec + bitstrings[bitstring]*this_vec
        return vec
    else:
        num_bits = len(bitstrings)
        vec = np.array(1.)
        for bit in bitstrings:
            vec = np.kron(vec,qubit_map[bit])
        return vec
def bitstrings_dict_from_vector(vec):
    num_bits = int(np.log2(len(vec)))
    state = dict()
    for i in range(len(vec)):
        if vec[i]!=0.:
            state[int_to_bitstring(i,num_bits)] = vec[i]
    return state

def bitstring_to_dict(bitstring):
    bitstrings = dict()
    num_bits = len(bitstring)
    for i in range(2**num_bits):
        bitstrings[int_to_bitstring(i,num_bits)] = 0
        bitstrings[bitstring] = 1.
    return bitstrings

def bitstring_from_dict(bitstrings):
    for bitstring in bitstrings:
        coeff=bitstrings[bitstring]
        match coeff:
            case 1.: return bitstring
            case -1.: return "-"+bitstring
            case 1.j: return "i"+bitstring
            case -1.j: return "-i"+bitstring
            case _: return "error - not a computational basis state"
    

def f_swaps(bitstring,swap_from,swap_to):
    num_swaps = len(swap_from)
    for i in range(num_swaps):
        temp = bitstring[swap_from[i]]
        #bitstring[swap_from[i]] = bitstring[swap_to[i]]
        bitstring = bitstring[:swap_from[i]] + bitstring[swap_to[i]] +bitstring[1+swap_from[i]:]
        #bitstring[swap_to[i]] = temp
        bitstring = bitstring[:swap_to[i]] + temp +bitstring[1+swap_to[i]:]
    return bitstring
def arbitrary_swap(num_qubits,swap_from,swap_to):
    num_swaps = len(swap_from)
    if len(swap_to) != num_swaps: return "error - num bits to swap must match"
    lgm = np.zeros((2**num_qubits,2**num_qubits))
    for bitint in range(2**num_qubits):
        transformed_bitstring = f_swaps(int_to_bitstring(bitint,num_qubits),swap_from,swap_to)
        row = np.array(1.)
        for bit in transformed_bitstring:
            row = np.kron(row,qubit_map[bit])
        lgm[bitint]=row
    return lgm
def arbitrary_U(U,bits,circuit_length):
    num_bits = len(bits)
    if(int(np.log2(np.shape(U)[0])) != num_bits): return "error matrix size does not match number of bits"
    total_U = U
    for i in range(circuit_length-num_bits):
        total_U = np.kron(total_U,np.eye(2))
    swap = arbitrary_swap(circuit_length,bits,list(range(len(bits))))
    return swap@total_U@swap

def controlled_U(U,num_qubits,controls,targets,control_val='1'):
    num_sub_qubits = len(targets)
    if(int(np.log2(np.shape(U)[0])) != num_sub_qubits): return "error matrix size does not match number of target bits"
    final_U = np.zeros((2**num_qubits,2**num_qubits))
    for bitint in range(2**num_qubits):
        bitstring=int_to_bitstring(bitint,num_qubits)
        swapped_bitstring = f_swaps(int_to_bitstring(bitint,num_qubits),targets,list(range(len(targets))))
        if all_bits_on(bitstring,controls,control_val):
            final_U = final_U + np.kron(U,projection_on_the_rest(swapped_bitstring[len(targets):]))
        else:
            final_U = final_U + np.kron(np.eye(2**len(targets)),projection_on_the_rest(swapped_bitstring[len(targets):]))
    swap = arbitrary_swap(num_qubits,targets,list(range(len(targets))))
    final_U = swap@final_U@swap
    return final_U/np.linalg.norm(final_U, axis=-1)[:, np.newaxis]

def controlled_on_bitstring(U,num_qubits,controls,targets,ctrl_bitstring):
    num_sub_qubits = len(targets)
    if(int(np.log2(np.shape(U)[0])) != num_sub_qubits): return "error matrix size does not match number of target bits"
    final_U = np.zeros((2**num_qubits,2**num_qubits))
    for bitint in range(2**num_qubits):
        bitstring=int_to_bitstring(bitint,num_qubits)
        swapped_bitstring = f_swaps(int_to_bitstring(bitint,num_qubits),targets,list(range(len(targets))))
        if all_bits_on(bitstring,controls,'1',control_bitstring=ctrl_bitstring):
            final_U = final_U + np.kron(U,projection_on_the_rest(swapped_bitstring[len(targets):]))
        else:
            final_U = final_U + np.kron(np.eye(2**len(targets)),projection_on_the_rest(swapped_bitstring[len(targets):]))
    swap = arbitrary_swap(num_qubits,targets,list(range(len(targets))))
    final_U = swap@final_U@swap
    normalization = np.linalg.norm(final_U, axis=-1)[:, np.newaxis]
    return final_U/normalization

def checkOperator(U):
    num_qubits=int(np.log2(np.shape(U)[0]))
    for i in range(2**num_qubits):
        bitstring=int_to_bitstring(i,num_qubits)
        transformed_bitstring = bitstring_from_dict(bitstrings_dict_from_vector(U@bitstrings_to_vector(bitstring)))
        print(bitstring + " -> " + transformed_bitstring)

In [8]:
z=np.array([[1.,0.],[0.,-1.]])
checkOperator(controlled_U(z,4,[1,2],[3]))

0000 -> 0000
0001 -> 0001
0010 -> 0010
0011 -> 0011
0100 -> 0100
0101 -> 0101
0110 -> 0110
0111 -> -0111
1000 -> 1000
1001 -> 1001
1010 -> 1010
1011 -> 1011
1100 -> 1100
1101 -> 1101
1110 -> 1110
1111 -> -1111


In [6]:

checkOperator(controlled_on_bitstring(z,4,[1,2],[3],'10'))

0000 -> 0000
0001 -> 0001
0010 -> 0010
0011 -> 0011
0100 -> 0100
0101 -> -0101
0110 -> 0110
0111 -> 0111
1000 -> 1000
1001 -> 1001
1010 -> 1010
1011 -> 1011
1100 -> 1100
1101 -> -1101
1110 -> 1110
1111 -> 1111


In [5]:
def all_bits_on(bitstring,controls,control_val='1',control_bitstring=''):
    if len(controls) == 0: return True
    if control_bitstring=='': control_bitstring=control_val*len(controls)
    for bit in range(len(controls)):
        if bitstring[controls[bit]] != control_bitstring[bit]:
            return False
    return True

def arbitrary_U(U,targets,circuit_size,controls=[],control_bitstring=''):
    if control_bitstring == '': control_bitstring = '1'*len(controls)
    num_sub_qubits = len(targets)
    if(int(np.log2(np.shape(U)[0])) != num_sub_qubits): return "error matrix size does not match number of target bits"
    final_U = np.zeros((2**circuit_size,2**circuit_size))
    for bitint in range(2**circuit_size):
        bitstring=int_to_bitstring(bitint,circuit_size)
        swapped_bitstring = f_swaps(int_to_bitstring(bitint,circuit_size),targets,list(range(len(targets))))
        if all_bits_on(bitstring,controls,'1',control_bitstring=control_bitstring):
            final_U = final_U + np.kron(U,projection_on_the_rest(swapped_bitstring[len(targets):]))
        else:
            final_U = final_U + np.kron(np.eye(2**len(targets)),projection_on_the_rest(swapped_bitstring[len(targets):]))
    swap = arbitrary_swap(circuit_size,targets,list(range(len(targets))))
    final_U = swap@final_U@swap
    normalization = np.linalg.norm(final_U, axis=-1)[:, np.newaxis]
    return final_U/normalization

In [14]:
checkOperator(arbitrary_U(z,[3],10))

0000000000 -> 0000000000
0000000001 -> 0000000001
0000000010 -> 0000000010
0000000011 -> 0000000011
0000000100 -> 0000000100
0000000101 -> 0000000101
0000000110 -> 0000000110
0000000111 -> 0000000111
0000001000 -> 0000001000
0000001001 -> 0000001001
0000001010 -> 0000001010
0000001011 -> 0000001011
0000001100 -> 0000001100
0000001101 -> 0000001101
0000001110 -> 0000001110
0000001111 -> 0000001111
0000010000 -> 0000010000
0000010001 -> 0000010001
0000010010 -> 0000010010
0000010011 -> 0000010011
0000010100 -> 0000010100
0000010101 -> 0000010101
0000010110 -> 0000010110
0000010111 -> 0000010111
0000011000 -> 0000011000
0000011001 -> 0000011001
0000011010 -> 0000011010
0000011011 -> 0000011011
0000011100 -> 0000011100
0000011101 -> 0000011101
0000011110 -> 0000011110
0000011111 -> 0000011111
0000100000 -> 0000100000
0000100001 -> 0000100001
0000100010 -> 0000100010
0000100011 -> 0000100011
0000100100 -> 0000100100
0000100101 -> 0000100101
0000100110 -> 0000100110
0000100111 -> 0000100111


In [192]:
import numpy as np

qubit_map = {'0':np.array([1,0]),'1':np.array([0,1])}    
x = np.array([[0.,1.],[1.,0.]])
y = np.array([[0.,-1.j],[1.j,0.]])
z = np.array([[1.,0.],[0.,-1.]])
h = 1/2**.5*np.array([[1.,1.],[1.,-1.]])
s = np.array([[1.,0.],[0.,1.j]])


def all_bits_on(bitstring,controls,control_bitstring=''):
    if len(controls) == 0: return True
    if control_bitstring=='': control_bitstring='1'*len(controls)
    for bit in range(len(controls)):
        if bitstring[controls[bit]] != control_bitstring[bit]:
            return False
    return True

def projection_on_the_rest(remaining_bitstring):
    product = np.array(1.)
    for bit in remaining_bitstring:
        product=np.kron(product,np.outer(qubit_map[bit],qubit_map[bit]))
    return product

def int_to_bitstring(i,length):
    bitstring = bin(i)[2:]
    bitlength = len(bitstring)
    if bitlength < length:
        for j in range(length-bitlength):
            bitstring = '0'+bitstring
    return bitstring

def bitstrings_to_vector(bitstrings):
    if type(bitstrings) == dict:
        num_bits = len(list(bitstrings)[0])
        vec = np.zeros(2**num_bits)
        for bitstring in bitstrings:
            this_vec = np.array(1.)
            for bit in bitstring:
                this_vec = np.kron(this_vec,qubit_map[bit])
            vec = vec + bitstrings[bitstring]*this_vec
        return vec
    else:
        num_bits = len(bitstrings)
        vec = np.array(1.)
        for bit in bitstrings:
            vec = np.kron(vec,qubit_map[bit])
        return vec
def bitstrings_dict_from_vector(vec):
    num_bits = int(np.log2(len(vec)))
    state = dict()
    for i in range(len(vec)):
        if vec[i]!=0.:
            state[int_to_bitstring(i,num_bits)] = vec[i]
    return state

def bitstring_to_dict(bitstring):
    bitstrings = dict()
    num_bits = len(bitstring)
    for i in range(2**num_bits):
        bitstrings[int_to_bitstring(i,num_bits)] = 0
        bitstrings[bitstring] = 1.
    return bitstrings

def state_as_string(bitstrings):
    final_bitstring=""
    for bitstring in bitstrings:
        coeff=np.round(bitstrings[bitstring],8)
        match coeff:
            case 1.: final_bitstring = bitstring
            case -1.: final_bitstring = "-"+bitstring
            case 1.j: final_bitstring = "i"+bitstring
            case -1.j: final_bitstring ="-i"+bitstring
            case 0.: pass
            case _: final_bitstring = final_bitstring + str(np.round(coeff,3))+"|"+bitstring+ "> + "
        
    if final_bitstring[-3:] ==" + ": return final_bitstring[:-3]
    else: return final_bitstring
    

def swap_bitstring(bitstring,swap_from,swap_to):
    num_swaps = len(swap_from)
    for i in range(num_swaps):
        temp = bitstring[swap_from[i]]
        bitstring = bitstring[:swap_from[i]] + bitstring[swap_to[i]] +bitstring[1+swap_from[i]:]
        bitstring = bitstring[:swap_to[i]] + temp +bitstring[1+swap_to[i]:]
    return bitstring

def arbitrary_swap(circuit_size,swap_from,swap_to):
    num_swaps = len(swap_from)
    if len(swap_to) != num_swaps: return "error - num bits to swap must match"
    lgm = np.zeros((2**circuit_size,2**circuit_size))
    for bitint in range(2**circuit_size):
        transformed_bitstring = swap_bitstring(int_to_bitstring(bitint,circuit_size),swap_from,swap_to)
        row = np.array(1.)
        for bit in transformed_bitstring:
            row = np.kron(row,qubit_map[bit])
        lgm[bitint]=row
    return lgm

def arbitrary_U(U,circuit_size,targets,controls=[],control_bitstring=''):
    num_sub_qubits = len(targets)
    if len(controls) == 0:
        if(int(np.log2(np.shape(U)[0])) != num_sub_qubits): return "error matrix size does not match number of bits"
        total_U = U
        for i in range(circuit_size-num_sub_qubits):
            total_U = np.kron(total_U,np.eye(2))
        swap = arbitrary_swap(circuit_size,targets,list(range(num_sub_qubits)))
        return swap@total_U@swap
    else:
        if control_bitstring == '': control_bitstring = '1'*len(controls)
        if(int(np.log2(np.shape(U)[0])) != num_sub_qubits): return "error matrix size does not match number of target bits"
        final_U = np.zeros((2**circuit_size,2**circuit_size))
        for bitint in range(2**circuit_size):
            bitstring=int_to_bitstring(bitint,circuit_size)
            swapped_bitstring = swap_bitstring(int_to_bitstring(bitint,circuit_size),targets,list(range(len(targets))))
            if all_bits_on(bitstring,controls,control_bitstring):
                final_U = final_U + np.kron(U,projection_on_the_rest(swapped_bitstring[len(targets):]))
            else:
                final_U = final_U + np.kron(np.eye(2**len(targets)),projection_on_the_rest(swapped_bitstring[len(targets):]))
        swap = arbitrary_swap(circuit_size,targets,list(range(len(targets))))
        final_U = swap@final_U@swap
        normalization = np.linalg.norm(final_U, axis=-1)[:, np.newaxis]
        return final_U/normalization

def checkOperator(U):
    num_qubits=int(np.log2(np.shape(U)[0]))
    for i in range(2**num_qubits):
        bitstring=int_to_bitstring(i,num_qubits)
        transformed_bitstring = bitstring_from_dict(bitstrings_dict_from_vector(U@bitstrings_to_vector(bitstring)))
        print(bitstring + " -> " + transformed_bitstring)
    

class QCircuit:

    def __init__(self,circuit_size,initial_state=None):
        self.circuit_size = circuit_size
        if initial_state is None: 
            self.statevector = bitstrings_to_vector('0'*circuit_size)
        else:self.statevector=initial_state/np.linalg.norm(initial_state)
        self.density_matrix = np.outer(self.statevector,self.statevector)
        self.unitary = np.eye(2**circuit_size)
    
    def apply_to_circuit(self,U):
        self.statevector = U@self.statevector
        self.density_matrix = U@self.density_matrix@U.conj().T
        self.unitary = self.unitary@U
    
    def arbitrary_U(self,U,targets,controls=[],control_bitstring=''):
        num_sub_qubits = len(targets)
        if control_bitstring !='' and len(controls) == 0: controls=list(range(len(control_bitstring)))
        if len(controls) == 0 and control_bitstring=='':
            if(int(np.log2(np.shape(U)[0])) != num_sub_qubits): return "error matrix size does not match number of bits"
            total_U = U
            for i in range(self.circuit_size-num_sub_qubits):
                total_U = np.kron(total_U,np.eye(2))
            swap = arbitrary_swap(self.circuit_size,targets,list(range(num_sub_qubits)))
            self.apply_to_circuit(swap@total_U@swap)
        else:
            if control_bitstring == '': control_bitstring = '1'*len(controls)
            if(int(np.log2(np.shape(U)[0])) != num_sub_qubits): return "error matrix size does not match number of target bits"
            final_U = np.zeros((2**self.circuit_size,2**self.circuit_size))
            for bitint in range(2**self.circuit_size):
                bitstring=int_to_bitstring(bitint,self.circuit_size)
                swapped_bitstring = swap_bitstring(int_to_bitstring(bitint,self.circuit_size),targets,list(range(len(targets))))
                if all_bits_on(bitstring,controls,control_bitstring):
                    final_U = final_U + np.kron(U,projection_on_the_rest(swapped_bitstring[len(targets):]))
                else:
                    final_U = final_U + np.kron(np.eye(2**len(targets)),projection_on_the_rest(swapped_bitstring[len(targets):]))
            swap = arbitrary_swap(self.circuit_size,targets,list(range(len(targets))))
            final_U = swap@final_U@swap
            normalization = np.linalg.norm(final_U, axis=-1)[:, np.newaxis]
            self.apply_to_circuit(final_U/normalization)
    
    def arbitrary_swap(self,swap_from,swap_to):
        num_swaps = len(swap_from)
        if len(swap_to) != num_swaps: return "error - num bits to swap must match"
        lgm = np.zeros((2**self.circuit_size,2**self.circuit_size))
        for bitint in range(2**self.circuit_size):
            transformed_bitstring = swap_bitstring(int_to_bitstring(bitint,self.circuit_size),swap_from,swap_to)
            row = np.array(1.)
            for bit in transformed_bitstring:
                row = np.kron(row,qubit_map[bit])
            lgm[bitint]=row
        self.apply_to_circuit(lgm)
    
    def x(self,bit):
        self.arbitrary_U(x,[bit])
    
    def y(self,bit):
        self.arbitrary_U(y,[bit])
        
    def z(self,bit):
        self.arbitrary_U(z,[bit])
    
    def h(self,bit):
        self.arbitrary_U(h,[bit])
        
    def s(self,bit):
        self.arbitrary_U(s,[bit])
        
    def p(self,bit,phi):
        self.arbitrary_U(np.array([[1.,0.],[0.,np.exp(phi*1.j)]]),[bit])    
    
    def reverse_bits(self):
        if self.circuit_size == 2:
            self.arbitrary_swap([0],[1])
        if self.circuit_size == 3:
            self.arbitrary_swap([0],[2])
        if self.circuit_size > 3:
            self.arbitrary_swap(list(range(self.circuit_size))[:int(self.circuit_size/2)],list(range(self.circuit_size))[int(np.ceil(self.circuit_size/2)):])

In [193]:
def grovers(goal_state, num_iterations):
    num_qubits = len(goal_state)+1
    circuit = QCircuit(num_qubits)
    
    # h on all except ancilla
    for qubit in range(num_qubits-1):
        circuit.h(qubit)
    # NOT on ancilla    
    circuit.x(num_qubits-1)
    # grover iteration
    for grover_iteration in range(num_iterations):
        # phase flip ancilla controlled by bitstring of goal state
        circuit.arbitrary_U(x,[num_qubits-1],control_bitstring=goal_state)
        
        # h on all except ancilla
        for qubit in range(num_qubits-1):
            circuit.h(qubit)

        # phase flip ancilla if state is all zeros
        circuit.arbitrary_U(z,[num_qubits-1],control_bitstring='0'*len(goal_state))

        # h on all except ancilla
        for qubit in range(num_qubits-1):
            circuit.h(qubit)
    return circuit

grovers_circuit = grovers('11010',int(2**2.5))
print(state_as_string(bitstrings_dict_from_vector(grovers_circuit.statevector)))

-0.034|000001> + -0.034|000011> + -0.034|000101> + -0.034|000111> + -0.034|001001> + -0.034|001011> + -0.034|001101> + -0.034|001111> + -0.034|010001> + -0.034|010011> + -0.034|010101> + -0.034|010111> + -0.034|011001> + -0.034|011011> + -0.034|011101> + -0.034|011111> + -0.034|100001> + -0.034|100011> + -0.034|100101> + -0.034|100111> + -0.034|101001> + -0.034|101011> + -0.034|101101> + -0.034|101111> + -0.034|110001> + -0.034|110011> + 0.677|110100> + -0.711|110101> + -0.034|110111> + -0.034|111001> + -0.034|111011> + -0.034|111101> + -0.034|111111>


In [303]:
def qft(circuit_size):
        
    circuit = QCircuit(circuit_size)
    
    # Begin the QFT
    for i in range(circuit_size):
        circuit.h(i)
        
        for k in range(2,circuit_size-i+1):
            R_k = np.array([[1.,0.],[0.,np.exp(1.j*2*np.pi/2**k)]])
            circuit.arbitrary_U(R_k,[i],controls=[i+k-1])
    circuit.reverse_bits()
    return circuit
    
initial_state = np.random.rand(2**3)
initial_state = initial_state/np.linalg.norm(initial_state)
qft_state = qft(3).unitary@initial_state
print(np.round(qft_state,3))

[ 0.871+0.j    -0.166-0.259j  0.049-0.049j -0.118+0.008j  0.125+0.j
 -0.118-0.008j  0.049+0.049j -0.166+0.259j]


In [304]:
def dft(x):
    N=len(x)
    y = np.repeat(0+0j,N)
    for k in range(N):
        for n in range(N):
            y[k] = y[k] + x[n]*np.exp(1j*2*np.pi*k*n/N)
    return y/(N**.5)
print(np.round(dft(initial_state),3))

[ 0.871+0.j    -0.166-0.259j  0.049-0.049j -0.118+0.008j  0.125-0.j
 -0.118-0.008j  0.049+0.049j -0.166+0.259j]


In [305]:
def qft_inverse(circuit_size):
        
    circuit = QCircuit(circuit_size)
    
    # Begin the QFT
    for i in range(circuit_size):
        circuit.h(i)
        
        for k in range(2,circuit_size-i+1):
            R_k = np.array([[1.,0.],[0.,np.exp(-1.j*2*np.pi/2**k)]])
            circuit.arbitrary_U(R_k,[i],controls=[i+k-1])
    circuit.reverse_bits()
    return circuit

print(initial_state)
print(np.round(qft_inverse(3).unitary@(qft(3).unitary@initial_state),3))

[0.18567049 0.07936189 0.12944297 0.19667685 0.58756172 0.37811468
 0.505693   0.39983246]
[0.186-0.j 0.079-0.j 0.129-0.j 0.197-0.j 0.588-0.j 0.378+0.j 0.506+0.j
 0.4  +0.j]


In [306]:


def QPE(U,num_auxiliary_bits,eigvec):
    
    m = int(np.log2(np.shape(U)[0]))
    num_qubits = num_auxiliary_bits + m
    circuit = QCircuit(num_qubits,initial_state=np.kron(bitstrings_to_vector('0'*num_auxiliary_bits),eigvec))
    for i in range(num_auxiliary_bits):
        circuit.h(i)
        for k in range(num_auxiliary_bits):
            circuit.arbitrary_U(U**(2**k),list(range(num_auxiliary_bits,num_qubits)),controls=[k])
            #circuit.arbitrary_U(U**(2**k),list(range(num_auxiliary_bits,num_qubits)),controls=[num_qubits-m-1-k])
    qfti_circuit = qft_inverse(num_auxiliary_bits)
    qfti_circuit.reverse_bits()
    circuit.arbitrary_U(qfti_circuit.unitary,list(range(num_auxiliary_bits)))
    return circuit

In [327]:

l = np.random.random()
test_gate = np.array([[1.,0.],[0.,np.exp(1.j*l)]])
eigval = np.linalg.eig(test_gate)[0][1]
eigvec = np.linalg.eig(test_gate)[1][:,1]
theta=np.log(eigval)/(2*np.pi*1.j)
theta


(0.11278252044608424+4.593723189255689e-19j)

In [328]:
qpe_circuit = QPE(test_gate,3,eigvec)

In [338]:
state=qpe_circuit.statevector
state = bitstrings_dict_from_vector(state)
state

{'0001': (-0.008275565659362211-0.007801059740749849j),
 '0011': (0.25607625032206505+0.5700453531518118j),
 '0101': (0.00428800933099353+0.1452810534548602j),
 '0111': (0.20599435367822916-0.542016345664126j),
 '1001': (-0.014017212465421553+0.014869821020967622j),
 '1011': (0.45185386787617815-0.20298217245258104j),
 '1101': (0.04139141760368907-0.0012216788128041195j),
 '1111': (0.06268887931362827+0.023825029042621404j)}

In [341]:
int('101',2)

5

In [342]:
5/8

0.625